In [ ]:
# the complete lambda function

import pandas as pd
import requests
from datetime import datetime
import pytz
import pymysql
import sqlalchemy as db
import json



def lambda_handler(event, context):
  schema="gans"
  host="wbs-project3-db.crfchknrpixs.eu-central-1.rds.amazonaws.com"
  user="kafawi"
  password="S0795735786s"
  port=3306
  con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
  aws_engine = db.create_engine(con, echo=True)
  aws_connection = aws_engine.connect()
  query = "select * from cities_and_populations"
  city_df = pd.read_sql(query, aws_connection)
  df = pd.DataFrame(city_df)

# creating dic for the weather 
  weather_dic = {"city_id":[],
              "country":[],
              "forcast_time":[],
              "temperature":[],
              "temperature_feels_like":[],
              "outlook":[],
              "outlook_description":[],
              "wind_speed":[],
              "wind_deg":[],
              "pressure":[],
              "humidity":[],
              "clouds":[],
              "rain":[],
              "snow":[]
              }

  for _, row in df.iterrows():
      API_key = "155ac89acb627b07f1c68cb640aef942"

  # making a loop for the cities
  
      city = row["name"]
      city_id = row["city_id"]
      weather_url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
      weather_response = requests.get(weather_url)
      weather_jason = weather_response.json()

      # getting the information fom the Jason
      for i in weather_jason["list"]:
          weather_dic["city_id"].append(city_id)
          weather_dic["country"].append(weather_jason["city"]["country"])
          weather_dic["forcast_time"].append(i["dt_txt"])
          weather_dic["temperature"].append(i["main"]["temp"])
          weather_dic["temperature_feels_like"].append(i["main"]["feels_like"])
          weather_dic["outlook"].append(i["weather"][0]["main"])
          weather_dic["outlook_description"].append(i["weather"][0]["description"])
          weather_dic["wind_speed"].append(i["wind"]["speed"])
          weather_dic["wind_deg"].append(i["wind"]["deg"])
          weather_dic["pressure"].append(i["main"]["pressure"])
          weather_dic["humidity"].append(i["main"]["humidity"])
          try:
              weather_dic["clouds"].append(i["clouds"]["all"])
          except:
              weather_dic["clouds"].append("0")
          try:
              weather_dic["snow"].append(i["snow"]["all"])
          except:
              weather_dic["snow"].append("0")
          try:
              weather_dic["rain"].append(i["rain"]["3h"])
          except:
              weather_dic["rain"].append("0")  
  weather_df = pd.DataFrame(weather_dic)


  

  weather_df.to_sql('weather',if_exists='append',con=aws_connection,index=False)
  aws_connection.commit()
  aws_connection.close()
  return {
  'statusCode': 200,
  'body': json.dumps('Hello from Lambda!')}